In [1]:
!pip install llama-index
!pip install cassio
!pip install nest-asyncio
!pip install sentence-transformers


  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.2 MB 3.3 MB/s eta 0:00:01
   ----------------- ---------------------- 0.5/1.2 MB 5.4 MB/s eta 0:00:01
   ------------------------------------ --- 1.1/1.2 MB 8.7 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 7.6 MB/s eta 0:00:00
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
   ---------------------------------------- 0.0/187.4 kB ? eta -:--:--
   ---------------------------------------- 187.4/187.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/367.8 kB ? eta -:--:--
   --------------------------------------- 367.8/367.8 kB 11.2 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
   ---------------------------------------- 0.0/191.0 kB ? et

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.1 which is incompatible.


In [16]:
# Grab a PDF from Arxiv for indexing
import requests

# The URL of the file you want to download
url = "https://arxiv.org/pdf/1706.03762.pdf"
# The local path where you want to save the file
file_path = "./attention.pdf"

# Perform the HTTP request
response = requests.get(url, timeout=30)

# Check if the request was successful
if response.status_code == requests.codes.ok:
    # Open the file in binary write mode and save the content
    with open(r"C:\Users\pavan\Downloads\what-is-air-pollution-2019.pdf", "wb") as file:
        file.write(response.content)
    print("Download complete.")
else:
    print("Error downloading the file.")

Download complete.


In [36]:
import nest_asyncio
nest_asyncio.apply()

import os
from getpass import getpass

os.environ["LLAMA_CLOUD_API_KEY"] = "llx-b5toQL69c8MZWasKLSTipIp6Rzrwf9rpYntsCKn5fnWZEg7X" 
os.environ["ASTRA_DB_API_ENDPOINT"] = "https://4d0ac63c-db4d-4461-a30b-5654bec200c2-us-east-2.apps.astra.datastax.com"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = "AstraCS:DHXHpyDSDzDEJgnCEmpMvMhU:bcabbd14b419dce9d8617a4408d370ac37be10d33c26720664744cc7956012a0"
# os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")


from llama_parse import LlamaParse

pdf_path = r"C:\Users\pavan\Downloads\GenAi.pdf"
documents = LlamaParse(result_type="text", verbose=True).load_data(pdf_path)

Started parsing the file under job_id 0d87c54d-d98d-4379-80fb-613a7e45eaed


In [37]:
# Take a quick look at some of the parsed text from the document:
documents[0].get_content()[10000:11000]

''

In [20]:
# !pip install llama_index.vector_stores.astra_db
import os

from llama_index.vector_stores.astra_db import AstraDBVectorStore

astra_db_store = AstraDBVectorStore(
    token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
    api_endpoint=os.environ["ASTRA_DB_API_ENDPOINT"],
    collection_name="demo11",
    embedding_dimension=1536,
)

In [34]:
from llama_index.core.node_parser import SentenceSplitter
default_splitter = SentenceSplitter()
split_nodes = default_splitter(documents)

In [39]:
# !pip install llama-index-embeddings-huggingface
!pip install llama_index.embeddings.fastembed 

     ---------------------------------------- 0.0/303.0 kB ? eta -:--:--
     -- ---------------------------------- 20.5/303.0 kB 640.0 kB/s eta 0:00:01
     -------- ---------------------------- 71.7/303.0 kB 975.2 kB/s eta 0:00:01
     ----------------------------------- -- 286.7/303.0 kB 2.5 MB/s eta 0:00:01
     -------------------------------------- 303.0/303.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/55.6 kB ? eta -:--:--
   ---------------------------------------- 55.6/55.6 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/62.5 kB ? eta -:--:--
   ---------------------------------------- 62.5/62.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/14.4 MB ? eta -:--:--
   - -------------------------------------- 0.4/14.4 MB 11.6 MB/s eta 0:00:02
   - -------------------------------------- 0.7/14.4 MB 8.5 MB/s eta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.1 which is incompatible.


In [22]:
from llama_index.llms.groq import Groq
from llama_index.core import StorageContext, VectorStoreIndex
# from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.fastembed import FastEmbedEmbedding
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-base-en-v1.5")
# from llama_index.llms import Groq
from llama_index.core import Settings

# Step 1: Initialize storage context with vector store (AstraDB)
storage_context = StorageContext.from_defaults(vector_store=astra_db_store)

# Step 2: Create the Groq LLM instance
groq_api_key = "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"
llm = Groq(api_key=groq_api_key, model="llama-3.1-70b-versatile")
print(type(Settings))

Settings.llm = llm
Settings.embed_model = embed_model
# embed_model = OpenAIEmbedding()
# # Step 3: Create settings with the LLM and embedding model
# settings = Settings(
#     llm=llm,
#     embed_model=embed_model,
#     chunk_size=512
# )

# Step 4: Create a vector store index with nodes and the updated settings
# index = VectorStoreIndex(nodes=nodes, storage_context=storage_context, settings=settings)

# Now, you can query or interact with the index


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

<class 'llama_index.core.settings._Settings'>


In [51]:
storage_context = StorageContext.from_defaults(vector_store=astra_db_store)

index = VectorStoreIndex.from_documents(
    documents=documents, storage_context=storage_context
)

ValueError: AstraDBVectorStore.add could not insert all requested documents (9 failed replace_one calls)